In [15]:
import os
from twitter_to_sqlite import utils
import pandas as pd
import time

In [6]:
auth = json.load(open("../auth/auth.json"))
owner_id = "143058191"
session = utils.session_for_auth(auth)

In [10]:
tweets_df = pd.read_csv("seen.csv")

In [8]:
c = "to_process"
url = f"https://api.twitter.com/1.1/collections/create.json?name={c}"
session.post(url)

<Response [200]>

In [9]:
url = f"https://api.twitter.com/1.1/collections/list.json?user_id={owner_id}"
response = session.get(url)
collections = response.json()["objects"]["timelines"]
for k in collections.keys():
    print(k, collections[k]["name"])
collections_list = [collections[k]["name"] for k in collections.keys()]

custom-1351839030959288320 to_process
custom-1351555076024893440 custom_newsfeed
custom-1351093543385899011 not_relevant


In [11]:
tweet_list = tweets_df["id"]

In [12]:
def proccecing_list(collection_id, tweet_list):
    
    collection_id = collection_id
    procc_list = []
    for counter, tweet_id in enumerate(tweet_list):
        if (counter+1) % 20 == 0:
            print(f"{(counter+1)} / {len(tweet_list)}")
        url = f"https://api.twitter.com/1.1/collections/entries/add.json?tweet_id={tweet_id}&id={collection_id}"
        response = session.post(url)
        if response.reason == "OK":
            errors = response.json()["response"]["errors"]
            if len(errors) > 0:
                procc_list.append({"tweet_id": tweet_id, "err_reason" : errors[0]["reason"]})
            else:
                procc_list.append({"tweet_id": tweet_id, "err_reason" : ""})
        else:
            print(counter)
            print(response.reason)
            break

    df = pd.DataFrame(procc_list)
    print(df["err_reason"].value_counts())
    return df

In [ ]:
# while respons is not ok = sleep for 60s


In [14]:
df = proccecing_list("custom-1351839030959288320", tweet_list.tolist())

20 / 1600
40 / 1600
60 / 1600
80 / 1600
100 / 1600
120 / 1600
140 / 1600
160 / 1600
180 / 1600
200 / 1600
220 / 1600
240 / 1600
260 / 1600
280 / 1600
300 / 1600
320 / 1600
340 / 1600
360 / 1600
380 / 1600
400 / 1600
420 / 1600
440 / 1600
460 / 1600
480 / 1600
500 / 1600
520 / 1600
540 / 1600
560 / 1600
580 / 1600
600 / 1600
620 / 1600
640 / 1600
660 / 1600
680 / 1600
700 / 1600
720 / 1600
740 / 1600
760 / 1600
770
Too Many Requests
                   571
not_found          189
protected_tweet     10
Name: err_reason, dtype: int64


In [21]:
df.shape

(770, 2)

In [22]:
df2 = proccecing_list("custom-1351839030959288320", tweet_list.tolist()[760:])

20 / 840
40 / 840
60 / 840
80 / 840
100 / 840
120 / 840
140 / 840
160 / 840
180 / 840
200 / 840
220 / 840
240 / 840
260 / 840
280 / 840
300 / 840
320 / 840
340 / 840
360 / 840
380 / 840
400 / 840
420 / 840
440 / 840
460 / 840
480 / 840
500 / 840
520 / 840
540 / 840
560 / 840
580 / 840
600 / 840
620 / 840
640 / 840
660 / 840
680 / 840
700 / 840
720 / 840
740 / 840
760 / 840
780 / 840
800 / 840
820 / 840
840 / 840
                   616
not_found          202
protected_tweet     14
duplicate            8
Name: err_reason, dtype: int64


In [23]:
df

,tweet_id,err_reason
0,1337875267096875015,
1,1335698027537969155,
2,1345699105385959435,not_found
3,1336874318882467842,
4,1337717825629253633,not_found
...,...,...
765,1336759256041840645,
766,755026645069750272,
767,1336457867235364867,
768,1339354804435341314,


In [24]:
df2

,tweet_id,err_reason
0,1340113559712350211,not_found
1,1337484424909209601,duplicate
2,1345334329698877441,duplicate
3,1339020019951349762,duplicate
4,1337920190340681729,not_found
...,...,...
835,1337184446789931008,not_found
836,1341250063876550656,
837,1336231355110322177,not_found
838,1344289183456768002,


In [26]:
frames = [df, df2]

In [27]:
result = pd.concat(frames)

In [31]:
result.drop_duplicates(subset="tweet_id",  inplace=True)

In [32]:
result

,tweet_id,err_reason
0,1337875267096875015,
1,1335698027537969155,
2,1345699105385959435,not_found
3,1336874318882467842,
4,1337717825629253633,not_found
...,...,...
835,1337184446789931008,not_found
836,1341250063876550656,
837,1336231355110322177,not_found
838,1344289183456768002,


In [34]:
result["err_reason"].value_counts()

                   1187
not_found           389
protected_tweet      24
Name: err_reason, dtype: int64

In [35]:
result.to_csv('seen.csv', index=False)